In [80]:
%matplotlib inline
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
import imblearn
import sys

import warnings
warnings.filterwarnings('ignore')




In [81]:
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')
train.head()

,protocol_type,service,flag,src_bytes,dst_bytes,hot,count,srv_count,diff_srv_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_serror_rate,class
0,icmp,20,2,491,0,0,2,2,0.00,150,25,0.17,0.03,0.17,0.00,normal
1,udp,45,2,146,0,0,13,1,0.15,255,1,0.00,0.60,0.88,0.00,normal
2,icmp,50,4,0,0,0,123,6,0.07,255,26,0.10,0.05,0.00,1.00,anomaly
3,icmp,25,2,232,8153,0,5,5,0.00,30,255,1.00,0.00,0.03,0.03,normal
4,icmp,25,2,199,420,0,30,32,0.00,255,255,1.00,0.00,0.00,0.00,normal


In [82]:
train.describe()

,service,flag,src_bytes,dst_bytes,hot,count,srv_count,diff_srv_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_serror_rate
count,25192.000000,25192.000000,2.519200e+04,2.519200e+04,25192.000000,25192.000000,25192.000000,25192.000000,25192.000000,25192.000000,25192.000000,25192.000000,25192.000000,25192.000000
mean,32.034217,2.581613,2.433063e+04,3.491847e+03,0.198039,84.591180,27.698754,0.062363,182.532074,115.063036,0.519791,0.082539,0.147453,0.285800
std,16.492717,1.145662,2.410805e+06,8.883072e+04,2.154202,114.673451,72.468242,0.178550,98.993895,110.646850,0.448944,0.187191,0.308367,0.445316
min,1.000000,1.000000,0.000000e+00,0.000000e+00,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,20.000000,2.000000,0.000000e+00,0.000000e+00,0.000000,2.000000,2.000000,0.000000,84.000000,10.000000,0.050000,0.000000,0.000000,0.000000
50%,25.000000,2.000000,4.400000e+01,0.000000e+00,0.000000,14.000000,8.000000,0.000000,255.000000,61.000000,0.510000,0.030000,0.000000,0.000000
75%,50.000000,4.000000,2.790000e+02,5.302500e+02,0.000000,144.000000,18.000000,0.060000,255.000000,255.000000,1.000000,0.070000,0.060000,1.000000
max,70.000000,11.000000,3.817091e+08,5.151385e+06,77.000000,511.000000,511.000000,1.000000,255.000000,255.000000,1.000000,1.000000,1.000000,1.000000


In [83]:
train['class'].value_counts()

normal     13449
anomaly    11743
Name: class, dtype: int64

In [84]:
type_attack = {'normal': 0, 'anomaly': 1}
train['class'] = train['class'].map(type_attack)
test['class'] = test['class'].map(type_attack)
train.head()

,protocol_type,service,flag,src_bytes,dst_bytes,hot,count,srv_count,diff_srv_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_serror_rate,class
0,icmp,20,2,491,0,0,2,2,0.00,150,25,0.17,0.03,0.17,0.00,0
1,udp,45,2,146,0,0,13,1,0.15,255,1,0.00,0.60,0.88,0.00,0
2,icmp,50,4,0,0,0,123,6,0.07,255,26,0.10,0.05,0.00,1.00,1
3,icmp,25,2,232,8153,0,5,5,0.00,30,255,1.00,0.00,0.03,0.03,0
4,icmp,25,2,199,420,0,30,32,0.00,255,255,1.00,0.00,0.00,0.00,0


In [85]:
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

In [86]:
le = preprocessing.LabelEncoder()
enc = OneHotEncoder()
lb = preprocessing.LabelBinarizer()

In [87]:
train['protocol_type'] = le.fit_transform(train['protocol_type'])
test['protocol_type'] = le.fit_transform(test['protocol_type'])
train.head()

,protocol_type,service,flag,src_bytes,dst_bytes,hot,count,srv_count,diff_srv_rate,dst_host_count,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_serror_rate,class
0,0,20,2,491,0,0,2,2,0.00,150,25,0.17,0.03,0.17,0.00,0
1,2,45,2,146,0,0,13,1,0.15,255,1,0.00,0.60,0.88,0.00,0
2,0,50,4,0,0,0,123,6,0.07,255,26,0.10,0.05,0.00,1.00,1
3,0,25,2,232,8153,0,5,5,0.00,30,255,1.00,0.00,0.03,0.03,0
4,0,25,2,199,420,0,30,32,0.00,255,255,1.00,0.00,0.00,0.00,0


In [88]:
datatrain = train.values
datatest = test.values

In [89]:
X = datatrain[:,0:15]
Y = datatrain[:,15]

In [90]:
W = datatest[:,0:15]
test_rez = datatest[:,15]

In [91]:
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)
test_df = min_max_scaler.fit_transform(W)

In [92]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X_scale, Y, test_size=0.3)

In [93]:
print(train_x.shape, test_x.shape, train_y.shape, test_y.shape, test_df.shape)

(17634, 15) (7558, 15) (17634,) (7558,) (10000, 15)


In [94]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

In [95]:
clf = MLPClassifier(hidden_layer_sizes=(100,100,100), max_iter=500, alpha=0.001,
                   solver='sgd', verbose=10, random_state=21, tol=0.000000001)

In [96]:
clf.fit(train_x, train_y)
clf.fit(test_x, test_y)
y_pred = clf.predict(test_df)

Iteration 1, loss = 0.68701248
Iteration 2, loss = 0.59900531
Iteration 3, loss = 0.51481240
Iteration 4, loss = 0.43069539
Iteration 5, loss = 0.36578016
Iteration 6, loss = 0.32265684
Iteration 7, loss = 0.29533101
Iteration 8, loss = 0.27594571
Iteration 9, loss = 0.26020841
Iteration 10, loss = 0.24715595
Iteration 11, loss = 0.23601847
Iteration 12, loss = 0.22637623
Iteration 13, loss = 0.21810769
Iteration 14, loss = 0.21130121
Iteration 15, loss = 0.20573793
Iteration 16, loss = 0.20095729
Iteration 17, loss = 0.19703356
Iteration 18, loss = 0.19352723
Iteration 19, loss = 0.19061141
Iteration 20, loss = 0.18796202
Iteration 21, loss = 0.18556083
Iteration 22, loss = 0.18336503
Iteration 23, loss = 0.18134083
Iteration 24, loss = 0.17946996
Iteration 25, loss = 0.17770717
Iteration 26, loss = 0.17608420
Iteration 27, loss = 0.17466557
Iteration 28, loss = 0.17329496
Iteration 29, loss = 0.17195026
Iteration 30, loss = 0.17063421
Iteration 31, loss = 0.16941359
Iteration 32, los

Iteration 253, loss = 0.06601643
Iteration 254, loss = 0.06620281
Iteration 255, loss = 0.06569551
Iteration 256, loss = 0.06596987
Iteration 257, loss = 0.06550450
Iteration 258, loss = 0.06538585
Iteration 259, loss = 0.06540288
Iteration 260, loss = 0.06496011
Iteration 261, loss = 0.06494987
Iteration 262, loss = 0.06473571
Iteration 263, loss = 0.06465992
Iteration 264, loss = 0.06440035
Iteration 265, loss = 0.06419107
Iteration 266, loss = 0.06422975
Iteration 267, loss = 0.06403919
Iteration 268, loss = 0.06411764
Iteration 269, loss = 0.06396317
Iteration 270, loss = 0.06407022
Iteration 271, loss = 0.06342631
Iteration 272, loss = 0.06408342
Iteration 273, loss = 0.06310704
Iteration 274, loss = 0.06347981
Iteration 275, loss = 0.06292928
Iteration 276, loss = 0.06302811
Iteration 277, loss = 0.06305561
Iteration 278, loss = 0.06276558
Iteration 279, loss = 0.06255111
Iteration 280, loss = 0.06263705
Iteration 281, loss = 0.06240757
Iteration 282, loss = 0.06234967
Iteration 

Iteration 2, loss = 0.67151556
Iteration 3, loss = 0.63652344
Iteration 4, loss = 0.60386169
Iteration 5, loss = 0.57053013
Iteration 6, loss = 0.53478027
Iteration 7, loss = 0.49745408
Iteration 8, loss = 0.46092921
Iteration 9, loss = 0.42734503
Iteration 10, loss = 0.39708522
Iteration 11, loss = 0.37150766
Iteration 12, loss = 0.35027869
Iteration 13, loss = 0.33332817
Iteration 14, loss = 0.31965598
Iteration 15, loss = 0.30836848
Iteration 16, loss = 0.29886194
Iteration 17, loss = 0.29063408
Iteration 18, loss = 0.28304957
Iteration 19, loss = 0.27623001
Iteration 20, loss = 0.26994362
Iteration 21, loss = 0.26399267
Iteration 22, loss = 0.25848794
Iteration 23, loss = 0.25326209
Iteration 24, loss = 0.24836021
Iteration 25, loss = 0.24374517
Iteration 26, loss = 0.23946256
Iteration 27, loss = 0.23538400
Iteration 28, loss = 0.23159340
Iteration 29, loss = 0.22796159
Iteration 30, loss = 0.22463183
Iteration 31, loss = 0.22156009
Iteration 32, loss = 0.21878169
Iteration 33, lo

Iteration 254, loss = 0.10432074
Iteration 255, loss = 0.10348595
Iteration 256, loss = 0.10313696
Iteration 257, loss = 0.10280069
Iteration 258, loss = 0.10271876
Iteration 259, loss = 0.10258728
Iteration 260, loss = 0.10210436
Iteration 261, loss = 0.10200315
Iteration 262, loss = 0.10206210
Iteration 263, loss = 0.10172085
Iteration 264, loss = 0.10150337
Iteration 265, loss = 0.10159571
Iteration 266, loss = 0.10098566
Iteration 267, loss = 0.10080540
Iteration 268, loss = 0.10046541
Iteration 269, loss = 0.10028292
Iteration 270, loss = 0.10013074
Iteration 271, loss = 0.09988718
Iteration 272, loss = 0.09975704
Iteration 273, loss = 0.09939471
Iteration 274, loss = 0.09916414
Iteration 275, loss = 0.09953077
Iteration 276, loss = 0.09886449
Iteration 277, loss = 0.09864851
Iteration 278, loss = 0.09850664
Iteration 279, loss = 0.09847254
Iteration 280, loss = 0.09821765
Iteration 281, loss = 0.09826632
Iteration 282, loss = 0.09771907
Iteration 283, loss = 0.09775292
Iteration 

In [97]:
from sklearn import metrics
from sklearn.metrics import classification_report,confusion_matrix

print("Model Accuracy:" "\n", accuracy_score(test_rez, y_pred))
print()
print("Confusion matrix:" "\n", confusion_matrix(test_rez, y_pred))
print()
print("Classification report:" "\n", classification_report(test_rez, y_pred)) 
print()

Model Accuracy:
 0.7505

Confusion matrix:
 [[4174  155]
 [2340 3331]]

Classification report:
               precision    recall  f1-score   support

         0.0       0.64      0.96      0.77      4329
         1.0       0.96      0.59      0.73      5671

    accuracy                           0.75     10000
   macro avg       0.80      0.78      0.75     10000
weighted avg       0.82      0.75      0.75     10000


